## Prog 4: 0915 Hrs Nifty_Sectors Status with LTP of the underlying sector stocks 

In [2]:
import requests
import pandas as pd
from datetime import datetime, time
import schedule
import time as time_module
import openpyxl
from openpyxl.styles import Font, PatternFill
import os

# Initialize session object
session = requests.Session()
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
}

def get_cookie():
    url = 'https://www.nseindia.com'
    response = session.get(url, headers=headers)
    cookies = response.cookies.get_dict()
    cookie_str = '; '.join([f"{key}={value}" for key, value in cookies.items()])
    return cookie_str

def fetch_data(url):
    if 'cookie' not in headers or headers['cookie'] is None:
        headers['cookie'] = get_cookie()
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
            print("Cookie expired. Fetching a new cookie...")
            headers['cookie'] = get_cookie()
            return fetch_data(url)
        else:
            print(f"HTTP Error {response.status_code}: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
    return None

def extract_data(positions, sector_name):
    data_list = []
    for item in positions['data']:
        symbol = item['symbol']
        ltp = clean_value(item['lastPrice'])
        p_change = clean_value(item['pChange'])
        open_price = clean_value(item['open'])
        day_low = clean_value(item['dayLow'])
        day_high = clean_value(item['dayHigh'])
        previous_close = clean_value(item['previousClose'])
        data_list.append([sector_name, symbol, ltp, p_change, open_price, day_low, day_high, previous_close])
    df = pd.DataFrame(data_list, columns=['sector', 'symbol', 'LTP', 'pChange', 'Open', 'D_Low', 'D_High', 'Pr.Close'])
    return df

def clean_value(value):
    if isinstance(value, str):
        value = value.replace(',', '')
        try:
            return float(value)
        except ValueError:
            return None
    return float(value)

def calculate_metrics(df):
    df['Daily Change'] = (df['LTP'] - df['Pr.Close']) / df['Pr.Close'] * 100
    return df

def fetch_and_process_data(urls):
    all_data = pd.DataFrame()
    sector_data = {}
    for sector_name, url in urls.items():
        data = fetch_data(url)
        if data:
            df = extract_data(data, sector_name)
            all_data = pd.concat([all_data, df])
            sector_data[sector_name] = df
    all_data = calculate_metrics(all_data)
    return all_data, sector_data

def remove_duplicates(df, key_col):
    if df.duplicated(subset=[key_col]).any():
        df = df.groupby(key_col).mean(numeric_only=True).reset_index()
    return df

def append_timestamp_columns(existing_df, new_df, key_col='symbol'):
    timestamp = datetime.now().strftime('%H_%M')
    existing_df = existing_df.copy()
    new_df = new_df.copy()

    # Remove duplicates
    new_df = remove_duplicates(new_df, key_col)
    existing_df = remove_duplicates(existing_df, key_col)

    new_df.set_index(key_col, inplace=True)
    existing_df.set_index(key_col, inplace=True, drop=False)

    # Add new columns with timestamp
    existing_df[f'LTP_{timestamp}'] = new_df['LTP']
    existing_df[f'pChange_{timestamp}'] = new_df['pChange']
    existing_df.reset_index(drop=True, inplace=True)

    return existing_df

def calculate_pchange_diff(existing_df, timestamp):
    pchange_col = f'pChange_{timestamp}'
    if pchange_col in existing_df.columns:
        pchange_columns = [col for col in existing_df.columns if col.startswith('pChange_') and col != pchange_col]
        
        if pchange_columns:
            pchange_columns.sort()
            previous_pchange_col = pchange_columns[-1]
            existing_df[f'Diff_{timestamp}'] = existing_df[pchange_col] - existing_df[previous_pchange_col]
    
    return existing_df

def save_to_csv(df, filename, key_col='symbol'):
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename)
        timestamp = datetime.now().strftime('%H_%M')
        existing_df = append_timestamp_columns(existing_df, df, key_col)
        existing_df = calculate_pchange_diff(existing_df, timestamp)
    else:
        existing_df = df

    # Round to 2 decimal places
    existing_df = existing_df.round(2)

    existing_df.to_csv(filename, index=False)

def merge_csv_files_to_excel(csv_files, excel_filename):
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        for csv_file in csv_files:
            if os.path.exists(csv_file):
                sheet_name = os.path.splitext(os.path.basename(csv_file))[0]
                df = pd.read_csv(csv_file)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                apply_styles(writer.sheets[sheet_name])
                apply_color_coding(writer.sheets[sheet_name])

def apply_color_coding(ws):
    pchange_columns = [cell.column for cell in ws[1] if cell.value and cell.value.startswith('pChange')]
    
    for col in pchange_columns:
        values = []
        for cell in ws.iter_cols(min_col=col, max_col=col, min_row=2):
            try:
                values.append(float(cell[0].value))
            except (TypeError, ValueError):
                continue
        if not values:
            continue
        
        min_value, max_value = min(values), max(values)
        range_value = max_value - min_value if max_value != min_value else 1
        
        for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col, max_col=col):
            for cell in row:
                try:
                    value = float(cell.value)
                    normalized_value = (value - min_value) / range_value
                    if value > 0:
                        shade = int(204 + 51 * normalized_value)
                        hex_color = f"{shade:02X}FF{shade:02X}"  # Green shades
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
                    elif value < 0:
                        shade = int(204 + 51 * abs(normalized_value))
                        hex_color = f"FF{shade:02X}{shade:02X}"  # Red shades
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
                except (TypeError, ValueError):
                    continue

def apply_styles(ws):
    # Apply styles to the header
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Apply filter to the first row
    ws.auto_filter.ref = ws.dimensions

    # Freeze the header row
    ws.freeze_panes = ws['E2']  # This will freeze the first row

def save_sector_data_to_csv_and_excel(sector_data):
    csv_files = []
    for sector, df in sector_data.items():
        csv_file = f"0.1_{sector.replace(' ', '_')}.csv"
        save_to_csv(df, csv_file)
        csv_files.append(csv_file)

    # Calculate and save sector summary
    sector_summary = pd.concat(sector_data.values()).groupby('sector').agg({'LTP': 'mean', 'pChange': 'mean'}).reset_index()
    sector_summary = sector_summary.round(2)  # Round to 2 decimal places
    save_to_csv(sector_summary, '0.1_nifty_Sector_Summary.csv', key_col='sector')
    csv_files.append('0.1_nifty_Sector_Summary.csv')

    # Save total analysis
    sector_total_analysis = pd.concat(sector_data.values())
    save_to_csv(sector_total_analysis, '0.1_nifty_Sector_Total_Analysis.csv')
    csv_files.append('0.1_nifty_Sector_Total_Analysis.csv')

    # Merge CSV files into one Excel workbook with different sheets
    merge_csv_files_to_excel(csv_files, '0.1_Nifty_SectorWise.xlsx')

def job():
    current_time = datetime.now().time()
    if (current_time >= time(9, 15) and current_time <= time(9, 45) and current_time.minute % 2 == 0) or \
       (current_time > time(9, 45) and current_time < time(10, 45) and current_time.minute % 3 == 0) or \
       (current_time > time(10, 45) and current_time < time(14, 0) and current_time.minute % 10 == 0) or \
       (current_time >= time(14, 0) and current_time <= time(15, 30) and current_time.minute % 2 == 0):
        
        urls = {
            'Nifty IT': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20IT',
            'Nifty Bank': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20BANK',
            'Nifty Pharma': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20PHARMA',
            'Nifty FMCG': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20FMCG',
            'Nifty Auto': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20AUTO',
            'Nifty Realty': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20REALTY',
            'Nifty Metal': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20METAL',
            'Nifty Media': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20MEDIA',
            'Nifty Energy': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20ENERGY',
            'Nifty PSU Bank': 'https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20PSU%20BANK'
        }

        all_data, sector_data = fetch_and_process_data(urls)

        # Save sector data to individual CSV files and merge into an Excel file
        save_sector_data_to_csv_and_excel(sector_data)

        # Print update message with timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f"The data in CSV and Excel for all sectors is updated at {timestamp}")

# Schedule the job to run every minute and handle timing within the job
schedule.every().minute.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time_module.sleep(1)


The data in CSV and Excel for all sectors is updated at 2024-08-09 09:57:11
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:00:12
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:00:13
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:03:17
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:03:19
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:06:21
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:06:23
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:09:25
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:09:27
Request Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:33:14
The data in CSV and Excel for all sectors is updated at 2024-08-09 10:33:15
The data in CSV and Excel for all sectors is updated at 

KeyboardInterrupt: 